# Symbolic Partial Derivative Routine

## Authors: Zach Etienne & Tyler Knowles

## This module contains a routine for computing an analytic partial derivative of a mathematical expression.

**Notebook Status:** <font color='red'><b> In progress </b></font>

**Validation Notes:** The module has been validated by comparing results to finite-difference derivative values in LALSuite.  <font color='red'><b> Add more info later </b></font>

<a id='intro'></a>

## Introduction
$$\label{intro}$$

Motivated by Mathematica being unable to generate readable partial derivatives of the SEOBNRv3 Hamiltonian, we wrote our own partial derivatve routine.

In [1]:
# Partial derivative routine
# Import necessary modules

import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import * #Check what is imported and remove *
import sympy as sp

bigstring = """
sigmaKerr0 = s1x + s2x
sigmaKerr1 = s1y + s2y
sigmaKerr2 = s1z + s2z
s1dots1 = s1x*s1x + s1y*s1y + s1z*s1z
s2dots2 = s2x*s2x + s2y*s2y + s2z*s2z
r2 = x*x + y*y + z*z
r = sp.sqrt(r2)
u = 1/r
tmppx = px - r
tmppy = py - r
tmppz = pz - r
"""

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1:](#step1) Split left- and right-hand sides of expressions
1. [Step 2:](#step2)

<a id='step1'></a>

# Step 1: Split lift- and right-hand sides of expressions \[Back to [top](#toc)\]
$$\label{step1}$$

We take the input string and split it first by line, then by "=".  Doing so 

In [2]:
# Split bigstring by carriage returns:
string_lines = bigstring.splitlines()

# Create "lr" array, which will store each left-hand side and right-hand side as strings.
lr = []
# Loop over each line in bigstring
for i in range(len(string_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(string_lines[i]) > 2 and string_lines[i][0] != "#":
        # Split each line by its equals sign.
        splitblah = blah[i].split("=")
        # Append to the "lr" array, removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #       (Lambda is a protected keyword):
        #lhs =
        lr.append(lhrh(lhs=splitblah[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=splitblah[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

<a id='step2'></a>

# Step 2: Insert title \[Back to [top](#toc)\]
$$\label{step2}$$

Insert description

In [3]:
xx = sp.Symbol('xx')
func = []
lhss = []
rhss = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs)(xx)))
#     print(i,lr[i].rhs)
    lhss.append(sp.sympify(lr[i].lhs))
    rhss.append(sp.sympify(lr[i].rhs))

# Next get a list of all the "free symbols" in the RHS expressions.
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for var in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(var)

full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [4]:
# Next declare input constants:
m1,m2,eta = sp.symbols("m1 m2 eta",real=True)
c0k2,c1k2,c0k3,c1k3,c0k4,c1k4,c2k4,c0k5,c1k5,c2k5 = sp.symbols("c0k2 c1k2 c0k3 c1k3 c0k4 c1k4 c2k4 c0k5 c1k5 c2k5",real=True)
KK,k5l,b3,bb3,d1,d1v2,dheffSS,dheffSSv2 = sp.symbols("KK k5l b3 bb3 d1 d1v2 dheffSS dheffSSv2",real=True)
tortoise,copysignresult = sp.symbols("tortoise copysignresult",real=True)
input_constants = [m1,m2,eta,
                   c0k2,c1k2,c0k3,c1k3,c0k4,c1k4,c2k4,c0k5,c1k5,c2k5,
                   KK,k5l,b3,bb3,d1,d1v2,dheffSS,dheffSSv2,
                   tortoise,copysignresult]

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [5]:
# Derivatives of input constants will always be zero, so
# print(full_symbol_list)
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb))(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    lhss_deriv.append(sp.sympify(str(lhss[i])+"prm"))
    newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))
    rhss_deriv.append(newrhs)
    #     rhss_deriv.append(sp.diff(rhss[i],xx))

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [6]:
def simplify_deriv(lhss_deriv,rhss_deriv):
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [7]:
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [8]:
def deriv_onevar(lhss_deriv,rhss_deriv,
                 xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0):

    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    for i in range(len(rhss_deriv_new)):
        for var in rhss_deriv_new[i].free_symbols:
            if str(var)=="xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,xprm)
            elif str(var)=="yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,yprm)
            elif str(var)=="zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,zprm)
            elif str(var)=="pxprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pxprm)
            elif str(var)=="pyprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pyprm)
            elif str(var)=="pzprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pzprm)
            elif str(var)=="s1xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1xprm)
            elif str(var)=="s1yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1yprm)
            elif str(var)=="s1zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1zprm)
            elif str(var)=="s2xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2xprm)
            elif str(var)=="s2yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2yprm)
            elif str(var)=="s2zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2zprm)
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [9]:
lhss_deriv_x,rhss_deriv_x = deriv_onevar(lhss_deriv,rhss_deriv, \
                            xprm=1,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_y,rhss_deriv_y = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=1,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_z,rhss_deriv_z = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=1,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_px,rhss_deriv_px = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=1,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_py,rhss_deriv_py = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_pz,rhss_deriv_pz = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=1,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1x,rhss_deriv_s1x = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=1,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1y,rhss_deriv_s1y = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=1,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1z,rhss_deriv_s1z = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=1,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s2x,rhss_deriv_s2x = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=1,s2yprm=0,s2zprm=0)
lhss_deriv_s2y,rhss_deriv_s2y = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=1,s2zprm=0)
lhss_deriv_s2z,rhss_deriv_s2z = deriv_onevar(lhss_deriv,rhss_deriv,
                            xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=1)

In [10]:
# CSE_results = sp.cse(rhss_deriv_px, sp.numbered_symbols("tmp"), order='canonical')
# for commonsubexpression in CSE_results[0]:
#     print("  "+str(commonsubexpression[0])+" = "+str(commonsubexpression[1]))
# for i,result in enumerate(CSE_results[1]):
#     print("rhss_deriv_px = "+str(result))

#for commonsubexpression in CSE_results[0]:

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [11]:
outstring = "/* SEOBNR Hamiltonian expression: */\n"
outstringsp = ""
outsplhs = []
outsprhs = []
for i in range(len(lr)):
    outstring += outputC(sp.sympify(lr[i].rhs),lr[i].lhs,"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
    outstringsp += lr[i].lhs+" = "+lr[i].rhs+"\n"
    outsplhs.append(sp.sympify(lr[i].lhs))
    outsprhs.append(sp.sympify(lr[i].rhs))
outstring += "\n\n\n/* SEOBNR \partial_x H expression: */\n"
for i in range(len(lhss_deriv_x)):
    outstring += outputC(rhss_deriv_x[i],str(lhss_deriv_x[i]),"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
    outstringsp += str(lhss_deriv_x[i])+" = "+str(rhss_deriv_x[i])+"\n"
    outsplhs.append(lhss_deriv_x[i])
    outsprhs.append(rhss_deriv_x[i])

<a id='stepn'></a>

# Step n: Insert title \[Back to [top](#toc)\]
$$\label{stepn}$$

Insert description

In [12]:
# for i in range(len(outsplhs)):
#     for j in range(i+1,len(outsplhs)):
#         outsprhs[j] = outsprhs[j].subs(outsplhs[i],outsprhs[i])
with open("/tmp/sympy_expression.py","w") as file:
    file.write("""
import sympy as sp
from outputC import *

m1,m2,x,y,z,px,py,pz,s1x,s1y,s1z,s2x,s2y,s2z,eta = sp.symbols("m1 m2 x y z px py pz s1x s1y s1z s2x s2y s2z eta",real=True)
c0k2,c1k2,c0k3,c1k3,c0k4,c1k4,c2k4,c0k5,c1k5,c2k5 = sp.symbols("c0k2 c1k2 c0k3 c1k3 c0k4 c1k4 c2k4 c0k5 c1k5 c2k5",real=True)
KK,k5l,b3,bb3,d1,d1v2,dheffSS,dheffSSv2 = sp.symbols("KK k5l b3 bb3 d1 d1v2 dheffSS dheffSSv2",real=True)
tortoise,copysignresult = sp.symbols("tortoise copysignresult",real=True)

""")
    for i in range(len(lr)):
        file.write(lr[i].lhs+" = "+"sp.symbols(\""+lr[i].lhs+"\")\n")
    file.write("\n")
    for i in range(len(lhss_deriv_x)):
        file.write(str(lhss_deriv_x[i]).replace("prm","prm_x")+" = "+str(rhss_deriv_x[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_x")+"\n")
    for i in range(len(lhss_deriv_y)):
        file.write(str(lhss_deriv_y[i]).replace("prm","prm_y")+" = "+str(rhss_deriv_y[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_y")+"\n")
    for i in range(len(lhss_deriv_z)):
        file.write(str(lhss_deriv_z[i]).replace("prm","prm_z")+" = "+str(rhss_deriv_z[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_z")+"\n")

    for i in range(len(lhss_deriv_px)):
        file.write(str(lhss_deriv_px[i]).replace("prm","prm_px")+" = "+str(rhss_deriv_px[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_px")+"\n")
    for i in range(len(lhss_deriv_py)):
        file.write(str(lhss_deriv_py[i]).replace("prm","prm_py")+" = "+str(rhss_deriv_py[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_py")+"\n")
    for i in range(len(lhss_deriv_pz)):
        file.write(str(lhss_deriv_pz[i]).replace("prm","prm_pz")+" = "+str(rhss_deriv_pz[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_pz")+"\n")

    for i in range(len(lhss_deriv_s1x)):
        file.write(str(lhss_deriv_s1x[i]).replace("prm","prm_s1x")+" = "+str(rhss_deriv_s1x[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s1x")+"\n")
    for i in range(len(lhss_deriv_s1y)):
        file.write(str(lhss_deriv_s1y[i]).replace("prm","prm_s1y")+" = "+str(rhss_deriv_s1y[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s1y")+"\n")
    for i in range(len(lhss_deriv_s1z)):
        file.write(str(lhss_deriv_s1z[i]).replace("prm","prm_s1z")+" = "+str(rhss_deriv_s1z[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s1z")+"\n")

    for i in range(len(lhss_deriv_s2x)):
        file.write(str(lhss_deriv_s2x[i]).replace("prm","prm_s2x")+" = "+str(rhss_deriv_s2x[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s2x")+"\n")
    for i in range(len(lhss_deriv_s2y)):
        file.write(str(lhss_deriv_s2y[i]).replace("prm","prm_s2y")+" = "+str(rhss_deriv_s2y[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s2y")+"\n")
    for i in range(len(lhss_deriv_s2z)):
        file.write(str(lhss_deriv_s2z[i]).replace("prm","prm_s2z")+" = "+str(rhss_deriv_s2z[i]).replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("Abs(","sp.Abs(").replace("prm","prm_s2z")+"\n")
    file.write("""
sp.cse(Hrealprm_x)
# outputC([Hrealprm_x,Hrealprm_y,Hrealprm_z,Hrealprm_px,Hrealprm_py,Hrealprm_pz,
#         Hrealprm_s1x,Hrealprm_s1y,Hrealprm_s1z,Hrealprm_s2x,Hrealprm_s2y,Hrealprm_s2z],
#        ["Hrealprm_x","Hrealprm_y","Hrealprm_z","Hrealprm_px","Hrealprm_py","Hrealprm_pz",
#         "Hrealprm_s1x","Hrealprm_s1y","Hrealprm_s1z","Hrealprm_s2x","Hrealprm_s2y","Hrealprm_s2z"],
#         "/tmp/outC.h","outCverbose=False,includebraces=False,SIMD_enable=True")
    """)

<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SEOBNR_Derivative_Routine.pdf](Tutorial-SEOBNR_Derivative_Routine.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [13]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("NRPyPN_shortcuts",location_of_template_file=os.path.join(".."))

Notebook output to PDF is only supported on Linux systems, with pdflatex installed.
